<a href="https://colab.research.google.com/github/masonritchotte/SoilRegression/blob/main/Narrowing_Search_prophet_w_cross_val_gridsearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import necessary libraries
import pandas as pd
from prophet import Prophet
from prophet.plot import plot_cross_validation_metric
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
import matplotlib.pyplot as plt
import itertools
import numpy as np
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

In [5]:
# Load the CSV file and adjust column names
file_path = 'aggregated_clean.csv'
data = pd.read_csv(file_path, header=None, names=['ds', 'y'])

# Convert the 'ds' column to datetime format
data['ds'] = pd.to_datetime(data['ds'])

In [ ]:
# Prep parameters for grid search
param_grid = {
    'changepoint_prior_scale': [0.001, 0.01],
    'seasonality_prior_scale': [0.0001, 0.001, 0.01],
    # Limit the grid due to dataset size; avoid overcomplex models
}

all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
mape_scores = []  # To store the MAPE scores for each parameter set

for params in all_params:
    # Initialize and fit the Prophet model with current parameters
    m = Prophet(**params)
    m.fit(data)

    # Perform cross-validation
    df_cv = cross_validation(m, initial='23 days', period='12 hours', horizon='6 hours')
    df_p = performance_metrics(df_cv)

    # Calculate and store the MAPE score
    mape = df_p['mape'].mean()
    mape_scores.append(mape)

# Identify the best performing set of parameters
best_score_index = np.argmin(mape_scores)
best_params = all_params[best_score_index]

print("Best Parameters:", best_params)

In [7]:
# Define the search space for hyperparameters
space = {
    'changepoint_prior_scale': hp.uniform('changepoint_prior_scale', 0.001, 0.1),
    'seasonality_prior_scale': hp.uniform('seasonality_prior_scale', 0.0001, 0.1)
}

def objective(params):
    m = Prophet(changepoint_prior_scale=params['changepoint_prior_scale'],
                seasonality_prior_scale=params['seasonality_prior_scale'])
    m.fit(data)

    df_cv = cross_validation(m, initial='23 days', period='12 hours', horizon='6 hours')
    df_p = performance_metrics(df_cv)
    mape = df_p['mape'].mean()

    # hyperopt minimizes the objective
    return {'loss': mape, 'status': STATUS_OK}

# Run the algorithm
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100, # You can decide how many steps to evaluate
            trials=trials)

print("Best Parameters:", best)


  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpoohc7yyo/1bp3xk78.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpoohc7yyo/0dorom4s.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=14325', 'data', 'file=/tmp/tmpoohc7yyo/1bp3xk78.json', 'init=/tmp/tmpoohc7yyo/0dorom4s.json', 'output', 'file=/tmp/tmpoohc7yyo/prophet_modelvfrgu7sr/prophet_model-20240320020355.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
02:03:55 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
02:03:57 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Making 15 forecasts with cutoffs between 2023-10-10 13:30:00 and 2023-10-17 13:30:00


  0%|          | 0/15 [00:00<?, ?it/s]

DEBUG:cmdstanpy:input tempfile: /tmp/tmpoohc7yyo/knnl399z.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpoohc7yyo/663h13a_.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=88891', 'data', 'file=/tmp/tmpoohc7yyo/knnl399z.json', 'init=/tmp/tmpoohc7yyo/663h13a_.json', 'output', 'file=/tmp/tmpoohc7yyo/prophet_modelfn5ec153/prophet_model-20240320020358.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
02:03:58 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
02:04:00 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpoohc7yyo/kgqv53jp.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpoohc7yyo/h6eaoroe.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/


  1%|          | 1/100 [00:35<59:12, 35.88s/trial, best loss: 0.33944568342574166]

...


 99%|█████████▉| 99/100 [57:51<00:35, 35.57s/trial, best loss: 0.2852131985225146]

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpoohc7yyo/fauwuild.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpoohc7yyo/8jkjuj2t.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=41134', 'data', 'file=/tmp/tmpoohc7yyo/fauwuild.json', 'init=/tmp/tmpoohc7yyo/8jkjuj2t.json', 'output', 'file=/tmp/tmpoohc7yyo/prophet_modelxhbmd0w9/prophet_model-20240320030146.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
03:01:46 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
03:01:47 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Making 15 forecasts with cutoffs between 2023-10-10 13:30:00 and 2023-10-17 13:30:00


  0%|          | 0/15 [00:00<?, ?it/s]

DEBUG:cmdstanpy:input tempfile: /tmp/tmpoohc7yyo/yq6yxg4r.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpoohc7yyo/ei8s60yu.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=48004', 'data', 'file=/tmp/tmpoohc7yyo/yq6yxg4r.json', 'init=/tmp/tmpoohc7yyo/ei8s60yu.json', 'output', 'file=/tmp/tmpoohc7yyo/prophet_modeloe7bmwae/prophet_model-20240320030149.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
03:01:49 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
03:01:49 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpoohc7yyo/lxd7jk9v.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpoohc7yyo/3erhq6lc.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/


100%|██████████| 100/100 [58:16<00:00, 34.97s/trial, best loss: 0.2852131985225146]
Best Parameters: {'changepoint_prior_scale': 0.0024956637537151757, 'seasonality_prior_scale': 0.0012132207404330075}
